In [1]:
import pickle
import scipy as sp
import igl
import robust_laplacian

import os.path as osp
import argparse
import numpy as np
import torch
import smplx
import glob
import cv2
import os
import pyrender
import trimesh
import json
from tqdm import tqdm

In [2]:
with open('../00122/Outer/Take9/SMPLX/mesh-f00011_smplx.pkl', 'rb') as file:
    smplx_data = pickle.load(file)
smplx_data

{'global_orient': array([ 0.00492813,  1.6237236 , -0.0699005 ], dtype=float32),
 'body_pose': array([-4.19760346e-02,  9.74555463e-02,  1.90463886e-01, -7.47566670e-02,
        -7.21446872e-02, -1.57268777e-01, -4.56898585e-02,  4.43277182e-03,
         5.37694916e-02, -6.97312132e-02,  8.66318271e-02,  9.14673135e-02,
        -1.26157841e-02, -6.44140169e-02, -1.91337466e-02,  5.14160208e-02,
        -1.75482202e-02,  8.12641811e-03,  1.33622453e-01,  5.47491647e-02,
        -1.69454706e-05,  1.37154654e-01, -3.88579182e-02,  1.61871758e-05,
         8.12471882e-02, -8.73514544e-03,  2.21590884e-02,  1.58502371e-05,
         1.57070062e-05,  1.54597165e-05,  1.54723148e-05,  1.39889626e-05,
        -1.51485538e-05, -1.13553563e-02, -4.63176444e-02, -6.14181906e-02,
        -5.00713438e-02,  2.19184086e-01, -1.77930251e-01, -2.58259997e-02,
        -1.98528439e-01,  1.84636593e-01,  7.81371593e-02, -1.03415944e-01,
         8.63157436e-02, -8.68193135e-02,  5.43496907e-02, -8.56566191

In [3]:
with open('/home/yeyiqi/Documents/repos/GaussianAvatar/assets/smpl_files/smplx/SMPLX_NEUTRAL.pkl', 'rb') as file:
    model_data = pickle.load(file, encoding='latin1')
model_data.keys()

dict_keys(['dynamic_lmk_bary_coords', 'hands_componentsl', 'ft', 'lmk_faces_idx', 'f', 'J_regressor', 'hands_componentsr', 'kintree_table', 'hands_coeffsr', 'joint2num', 'hands_meanl', 'lmk_bary_coords', 'weights', 'posedirs', 'dynamic_lmk_faces_idx', 'part2num', 'vt', 'hands_meanr', 'hands_coeffsl', 'v_template', 'shapedirs'])

In [4]:
body_model_param = {
    'betas': np.expand_dims(smplx_data['betas'],0),
    'expression': np.expand_dims(smplx_data['expression'],0),
    'transl': np.expand_dims(smplx_data['transl'],0),
    'global_orient': np.expand_dims(smplx_data['global_orient'], 0),
    'body_pose': np.expand_dims(smplx_data['body_pose'], 0),
    'jaw_pose': np.expand_dims(smplx_data['jaw_pose'], 0),
    'leye_pose': np.expand_dims(smplx_data['leye_pose'], 0),
    'reye_pose': np.expand_dims(smplx_data['reye_pose'], 0),
    'left_hand_pose': np.expand_dims(smplx_data['left_hand_pose'], 0),
    'right_hand_pose': np.expand_dims(smplx_data['right_hand_pose'], 0)
}

In [5]:
body_model_param_tensor = {
    key: torch.tensor(body_model_param[key]) for key in body_model_param.keys()
}

In [6]:
v_template = torch.from_numpy(model_data['v_template'])
shapedirs = torch.from_numpy(model_data['shapedirs'])
posedirs = torch.from_numpy(model_data['posedirs'])
J_regressor = torch.from_numpy(model_data['J_regressor'])
parents = torch.from_numpy(model_data['kintree_table'])
weights = torch.from_numpy(model_data['weights'])
parents = parents[0]
parents[0] = -1
parents

tensor([-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14,
        16, 17, 18, 19, 15, 15, 15, 20, 25, 26, 20, 28, 29, 20, 31, 32, 20, 34,
        35, 20, 37, 38, 21, 40, 41, 21, 43, 44, 21, 46, 47, 21, 49, 50, 21, 52,
        53])

In [7]:
model_folder = '/home/yeyiqi/Documents/repos/GaussianAvatar/assets/smpl_files'
model_type = 'smplx'

kwargs = dict(
        gender='neutral',
        num_betas=10,
        use_face_contour=True,
        num_pca_comps=12,
        use_pca=True,
        batch_size=1,
        ext = 'pkl'
)
model = smplx.create(
    model_path = model_folder,
    model_type = model_type,
    **kwargs
)

/home/yeyiqi/.conda/envs/MyFirst/lib/python3.10/site-packages/smplx/body_models.py:1009: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  dynamic_lmk_bary_coords = torch.tensor(


In [8]:
output = model(**body_model_param_tensor)

torch.Size([1, 55, 3, 3])


In [9]:
pose1_tsm = output.transform_matrix
pose1_tsm = pose1_tsm.squeeze()
pose1_tsm.shape

torch.Size([55, 4, 4])

In [10]:
with open('../00122/Outer/Take9/SMPLX/mesh-f00090_smplx.pkl', 'rb') as file:
    smplx_data2 = pickle.load(file)
smplx_data2

{'global_orient': array([-0.05113928,  1.0175937 , -0.0503974 ], dtype=float32),
 'body_pose': array([ 1.33996606e-01,  2.34585330e-01,  2.02095106e-01, -2.09829733e-01,
         1.08767666e-01, -5.03316559e-02,  1.23106748e-01, -3.99900436e-01,
        -1.47038465e-02,  5.24092950e-02,  2.23365456e-01,  1.33410335e-01,
         9.62239653e-02,  1.26511410e-01,  2.01036711e-03,  5.99068105e-02,
        -3.24472427e-01,  2.22415533e-02,  1.26188725e-01,  3.26585501e-01,
        -1.73083372e-05,  1.31123200e-01,  1.89871371e-01,  1.78726132e-05,
        -4.11981717e-02, -2.93310344e-01,  3.28465551e-02,  1.53884012e-05,
         1.64952598e-05,  1.53351175e-05,  1.45459526e-05,  1.51263275e-05,
        -1.50468722e-05,  5.27108535e-02,  2.33393312e-01, -4.94126752e-02,
        -1.57214791e-01, -8.92584324e-02,  3.24505717e-01, -2.01267511e-01,
        -2.26869076e-01, -5.73507957e-02,  1.31839737e-02,  2.62128353e-01,
        -8.07462484e-02, -4.62427773e-02, -3.90639484e-01, -1.04103014

In [11]:
body_model_param2 = {
    'betas': np.expand_dims(smplx_data2['betas'],0),
    'expression': np.expand_dims(smplx_data2['expression'],0),
    'transl': np.expand_dims(smplx_data2['transl'],0),
    'global_orient': np.expand_dims(smplx_data2['global_orient'], 0),
    'body_pose': np.expand_dims(smplx_data2['body_pose'], 0),
    'jaw_pose': np.expand_dims(smplx_data2['jaw_pose'], 0),
    'leye_pose': np.expand_dims(smplx_data2['leye_pose'], 0),
    'reye_pose': np.expand_dims(smplx_data2['reye_pose'], 0),
    'left_hand_pose': np.expand_dims(smplx_data2['left_hand_pose'], 0),
    'right_hand_pose': np.expand_dims(smplx_data2['right_hand_pose'], 0)
}

In [12]:
body_model_param_tensor2 = {
    key: torch.tensor(body_model_param[key]) for key in body_model_param.keys()
}

In [13]:
output2 = model(**body_model_param_tensor2)

torch.Size([1, 55, 3, 3])


In [14]:
pose2_tsm = output2.transform_matrix
pose2_tsm = pose2_tsm.squeeze()
pose2_tsm.shape

torch.Size([55, 4, 4])

In [15]:
from plyfile import PlyData
cloth = PlyData.read('./outer.ply')
cloth.elements

(PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar'), PlyProperty('alpha', 'uchar'), PlyProperty('u', 'float'), PlyProperty('v', 'float')), count=15683, comments=[]),
 PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'int'),), count=29318, comments=[]))

In [16]:
cloth_vertices_data = cloth['vertex'].data
cloth_vertices = np.array([
    (v['x'], v['y'], v['z'])
    for v in cloth_vertices_data
], dtype=np.float32)
cloth_vertices_tensor = torch.from_numpy(cloth_vertices)
cloth_vertices_tensor.shape

torch.Size([15683, 3])

In [17]:
cloth_face_data = cloth['face'].data
cloth_faces = np.array([
    list(f['vertex_indices'])
    for f in cloth_face_data
], dtype=np.int32)
cloth_faces_tensor = torch.from_numpy(cloth_faces)
cloth_faces_tensor.shape

torch.Size([29318, 3])

In [18]:
cloth_weights = np.load('outer_weights.npy')
cloth_weights = torch.from_numpy(cloth_weights)
cloth_weights.shape

torch.Size([15683, 55])

In [19]:
R1 = pose1_tsm[:, :3, :3]
t1 = pose1_tsm[:, :3, 3]
R1_inv = torch.transpose(R1, 1, 2)
t1_inv = -torch.bmm(R1_inv, t1.unsqueeze(-1)).squeeze(-1)

T1_inv = torch.eye(4).repeat(pose1_tsm.shape[0], 1, 1)
T1_inv[:, :3, :3] = R1_inv
T1_inv[:, :3, 3] = t1_inv

T1_to_T2 = torch.bmm(pose2_tsm, T1_inv)
T1_to_T2.shape

torch.Size([55, 4, 4])

In [20]:
cloth_homogenous_verts = torch.cat([
    cloth_vertices_tensor,
    torch.ones(cloth_vertices_tensor.shape[0],1)
], dim=1)
cloth_homogenous_verts

tensor([[-0.3922,  1.2456, -0.3545,  1.0000],
        [-0.3902,  1.2463, -0.3555,  1.0000],
        [-0.3871,  1.2463, -0.3567,  1.0000],
        ...,
        [-0.1475,  1.1942,  0.1961,  1.0000],
        [-0.4317,  1.6810, -0.0901,  1.0000],
        [-0.2270,  1.7640,  0.2341,  1.0000]])

In [21]:
transformed_verts = torch.zeros_like(cloth_homogenous_verts)
for j in range(T1_to_T2.shape[0]):
    weight = cloth_weights[:, j].unsqueeze(-1)
    verts_j = cloth_homogenous_verts @ T1_to_T2[j].T
    transformed_verts += weight * verts_j
transformed_verts = transformed_verts[:, :3]
transformed_verts.shape

torch.Size([15683, 3])

In [22]:
transformed_verts = transformed_verts.numpy()
transformed_verts

array([[-0.39217803,  1.2455803 , -0.35448405],
       [-0.39018205,  1.2462603 , -0.35546705],
       [-0.38706604,  1.2463005 , -0.3566731 ],
       ...,
       [-0.14750497,  1.1942499 ,  0.19611496],
       [-0.43171102,  1.6810005 , -0.09013478],
       [-0.22701296,  1.7639604 ,  0.23412089]], dtype=float32)

In [23]:
cloth_faces

array([[    4,     2,     1],
       [    2,     4,     5],
       [    2,     5,     6],
       ...,
       [  721,   634,   722],
       [11703, 12022, 11708],
       [11709, 11708, 12022]], dtype=int32)

In [24]:
with open('cloth_lbs.ply', 'w') as file:
    file.write(f'ply\n')
    file.write(f"format ascii 1.0\n")
    file.write(f"element vertex {len(transformed_verts)}\n")
    file.write(f"property float x\n")
    file.write(f"property float y\n")
    file.write(f"property float z\n")
    file.write(f"element face {len(cloth_faces)}\n")
    file.write(f"property list uchar int vertex_indices\n")
    file.write(f"end_header\n")
    for i in range(len(transformed_verts)):
        vertex = transformed_verts[i]
        file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")
    for face in cloth_faces:
        file.write(f"3 {face[0]} {face[1]} {face[2]}\n")

In [25]:
print("vertex has NaN:", np.isnan(transformed_verts).any())

vertex has NaN: True
